In [1]:
import autogen
from autogen.agentchat.contrib.agent_builder import AgentBuilder

from IPython import get_ipython
from typing_extensions import Annotated


config_file_or_env = "/workspaces/autogen/OAI_CONFIG_LIST.json"


def exec_python(cell: Annotated[str, "Valid Python cell to execute."]) -> str:
    ipython = get_ipython()
    result = ipython.run_cell(cell)
    log = str(result.result)
    if result.error_before_exec is not None:
        log += f"\n{result.error_before_exec}"
    if result.error_in_exec is not None:
        log += f"\n{result.error_in_exec}"
    return log


config_list = autogen.config_list_from_json(config_file_or_env, filter_dict={"model": ["gpt-4-1106-preview", "gpt-4"]})


llm_config = {
    "config_list": config_list,
    "timeout": 120,
}


def start_task(execution_task: str, agent_list: list):
    group_chat = autogen.GroupChat(agents=agent_list, messages=[], max_round=12)
    manager = autogen.GroupChatManager(groupchat=group_chat, llm_config={"config_list": config_list})
    agent_list[0].initiate_chat(manager, message=execution_task)


builder = AgentBuilder(
    config_file_or_env=config_file_or_env, builder_model="gpt-4-1106-preview", agent_model="gpt-4-1106-preview"
)

building_task = (
    "Draw two agents chatting with each other with an example dialog using matplotlib. Don't add plt.show()."
)

list_of_functions = [
    {
        "name": "exec_python",
        "description": "run cell in ipython and return the execution result.",
        "function": exec_python,
    }
]

agent_list, agent_configs = builder.build(building_task, llm_config, list_of_functions)

ModuleNotFoundError: No module named 'autogen.agentchat.contrib.agent_builder'

In [ ]:
start_task(execution_task=building_task, agent_list=agent_list)

In [ ]:
builder.clear_all_agents(recycle_endpoint=True)

saved_path = builder.save()